# NBA Win Margin Data Analysis

This Study utilizes NBA game data to test if we can significantly improve upon NBA lines generated by sportsbooks.



### Betting on NBA games

Sports books require customers to bet at odds of -110.  That means that betters must lay 110 units to win 100.  This ratio requires the better to win 52.38% of their bets to break even betting against the house. 

>>>$ .5238 * 100 =  .4762 * 110 $


This analysis attempts to use past NBA box score data to create a subset of predictions that improve on the 52.38% accuracy against the spread on unseen data.  

In this first notebook the data is transformed so that no information about the game that is unknown before the game is included in the analysis.  Historical moving averages and cumulative season data are computed and game data is merged so that each row is a single game.


In [1]:
import pandas as pd
from datetime import datetime

%matplotlib inline

In [2]:
nba2006 = pd.read_csv('./data/2006-2007_NBA_Box_Score_Team_Stats.csv')
nba2007 = pd.read_csv('./data/2007-2008_NBA_Box_Score_Team_Stats.csv')
nba2008 = pd.read_csv('./data/2008-2009_NBA_Box_Score_Team_Stats.csv')
nba2009 = pd.read_csv('./data/2009-2010_NBA_Box_Score_Team_Stats.csv')
nba2010 = pd.read_csv('./data/2010-2011_NBA_Box_Score_Team_Stats.csv')
nba2011 = pd.read_csv('./data/2011-2012_NBA_Box_Score_Team_Stats.csv')
nba2012 = pd.read_csv('./data/2012-2013_NBA_Box_Score_Team_Stats.csv')
nba2013 = pd.read_csv('./data/2013-2014_NBA_Box_Score_Team-Stats.csv')
nba2014 = pd.read_csv('./data/2014-2015_NBA_Box_Score_Team-Stats.csv')
#nba2015 = pd.read_csv('./data/2015-2016_NBA_Box_Score_Team-Stats.csv')
#nba2016 = pd.read_csv('./data/2016-2017_NBA_Box_Score_Team-Stats.csv')
#nba2017 = pd.read_csv('./data/2017-2018_NBA_Box_Score_Team-Stats.csv')

A couple of column names changed in 2014

In [3]:
nba2006.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2619 entries, 0 to 2618
Data columns (total 50 columns):
DATASET             2618 non-null object
DATE                2618 non-null object
TEAMS               2618 non-null object
VENUE               2618 non-null object
1Q                  2618 non-null float64
2Q                  2618 non-null float64
3Q                  2618 non-null float64
4Q                  2618 non-null float64
OT1                 180 non-null float64
OT2                 34 non-null float64
OT3                 2 non-null float64
OT4                 0 non-null float64
F                   2618 non-null float64
MIN                 2618 non-null float64
FG                  2618 non-null float64
FGA                 2618 non-null float64
3P                  2618 non-null float64
3PA                 2618 non-null float64
FT                  2618 non-null float64
FTA                 2618 non-null float64
OR                  2618 non-null float64
DR                  2618

In [4]:
nba2013.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2638 entries, 0 to 2637
Data columns (total 51 columns):
DATASET             2638 non-null object
DATE                2638 non-null object
TEAMS               2638 non-null object
VENUE               2638 non-null object
1Q                  2638 non-null int64
2Q                  2638 non-null int64
3Q                  2638 non-null int64
4Q                  2638 non-null int64
OT1                 176 non-null float64
OT2                 28 non-null float64
OT3                 6 non-null float64
OT4                 0 non-null float64
F                   2638 non-null int64
MIN                 2638 non-null float64
FG                  2638 non-null int64
FGA                 2638 non-null int64
3P                  2638 non-null int64
3PA                 2638 non-null int64
FT                  2638 non-null int64
FTA                 2638 non-null int64
OR                  2638 non-null int64
DR                  2638 non-null int64
TOT     

In [5]:
diff = {'OPENING SPREAD' : 'spread',
        'OPENING TOTAL'  : 'total',
        'CLOSING ODDS'   : 'closing',
        'Unnamed: 36'    : 'unnamed: 35',
        'Unnamed: 37'    : 'unnamed: 36',
        'Unnamed: 38'    : 'unnamed: 37',
        'Unnamed: 39'    : 'unnamed: 38'
       }

nba2013.rename(columns = diff, inplace = True)
nba2014.rename(columns = diff, inplace = True)

nba2013.drop(columns = ['TO TO'], inplace =True)
nba2014.drop(columns = ['TO TO'], inplace = True)

In [6]:
nba2014.head()

,DATASET,DATE,TEAMS,VENUE,1Q,2Q,3Q,4Q,OT1,OT2,...,CREW REFEREES,OPENING ODDS,spread,total,MOVEMENTS,closing,MONEYLINE,HALFTIME,BOX SCORE,ODDS
0,2014-2015 Regular Season,10/28/2014,Orlando,Road,25,16,23,20,NaN,NaN,...,David Jones,196.0,9.5,196.0,191 / 190.5 / 191.5,191.0,380,94.5,NaN,NaN
1,2014-2015 Regular Season,10/28/2014,New Orleans,Home,24,24,30,23,NaN,NaN,...,Dedric Taylor,-9.5,-9.5,196.0,-10.5 / -9.5 / -9 -05,-9.0,-475,-3.5,Boxscore,Odds
2,2014-2015 Regular Season,10/28/2014,Dallas,Road,24,29,20,27,NaN,NaN,...,Pat Fraher,206.5,6.0,206.5,203.5 / 203.5u14 / 203,203.5,140,102,NaN,NaN
3,2014-2015 Regular Season,10/28/2014,San Antonio,Home,26,19,31,25,NaN,NaN,...,Haywoode Workman,-6.0,-6.0,206.5,-4 -05 / -4 -07 / -3.5 -15,-3.5,-165,-4,Boxscore,Odds
4,2014-2015 Regular Season,10/28/2014,Houston,Road,31,31,23,23,NaN,NaN,...,Courtney Kirkland,-6.5,-6.5,209.0,-6.5 / -7 / -7 -09,-7.0,-300,+1.5 -20,NaN,NaN


In [7]:
nba2006.head(10).T

,0,1,2,3,4,5,6,7,8,9
DATASET,2006-2007 Regular Season,2006-2007 Regular Season,2006-2007 Regular Season,2006-2007 Regular Season,2006-2007 Regular Season,2006-2007 Regular Season,2006-2007 Regular Season,2006-2007 Regular Season,2006-2007 Regular Season,2006-2007 Regular Season
DATE,10/31/06,10/31/06,10/31/06,10/31/06,11/01/06,11/01/06,11/01/06,11/01/06,11/01/06,11/01/06
TEAMS,Chicago,Miami,Phoenix,LA Lakers,Indiana,Charlotte,Chicago,Orlando,Atlanta,Philadelphia
VENUE,Road,Home,Road,Home,Road,Home,Road,Home,Road,Home
1Q,22,16,41,26,23,27,20,32,14,29
2Q,37,14,17,27,26,23,25,31,21,20
3Q,21,21,21,34,28,23,25,23,23,24
4Q,28,15,27,27,29,26,24,23,17,15
OT1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
OT2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


### NBA csv File Structure

Currently, the NBA data has two records for each game one with the home team data and one with the away team data.  The unit of analysis for this study is the game so the data needs to be rearranged to use the game as the unit of analysis.  Therefore, each even row must be appended to the odd row above it to make the game the unit of analysis

In [8]:


def merge_home_away(df):
    
    
  
    #make all column names lower snake case
    df.columns = [col.lower().replace(' ', '_') for col in df.columns]
    
    #drop any empty data 
    df.date.dropna(inplace = True)
    #replace missing data with zeros
    missing_list = ['ot1', 'ot2', 'ot3', 'ot4']
    
    
    #some missing moneylines going to set them to zero for now look explore later
    df['moneyline'].fillna(0, inplace = True)
    df['movements'].fillna('none', inplace = True)
    
    
    for ot in missing_list:
        df[ot].fillna(0, inplace = True)
    
    
    #split up the rows
    df_road =  df[df.index %2 == 0]
    df_home =  df[df.index %2 == 1] 
    
    df.drop(columns = ['box_score', 
                       'odds',
                       'venue',
                       'halftime',
                       'opening_odds',
                       'movements',
                       'poss',
                       'closing'], inplace = True)
    
    #drop unnessecary columns in df1
    
    df_home.drop(columns = ['main_referee'
                        
                           ], inplace = True)
    
    #drop redundant columns in df2
    df_road.drop(columns = ['dataset',
                            'date',
                            'pts',
                            'total',
                            'moneyline',
                            'spread'
                           ], inplace = True)
    
    #rename columns df1 and df2 
    
    home =  {'starting_lineup' : 'home_starter1',
             'unnamed:_35'     : 'home_starter2',
             'unnamed:_36'     : 'home_starter3',
             'unnamed:_37'     : 'home_starter4',
             'unnamed:_38'     : 'home_starter5',
             'f'               : 'home_score'
            }
   
    df_home.rename(columns = home, inplace = True) 
    
    away =  {'teams'     : 'away_team',
             '1q'        : 'away_1q',
             '2q'        : 'away_2q',
             '3q'        : 'away_3q',
             '4q'        : 'away_4q',
             'ot1'       : 'away_ot1',
             'ot2'       : 'away_ot2',
             'ot3'       : 'away_ot3',
             'ot4'       : 'away_ot4',
             'f'         : 'away_score',
             'min'       : 'away_min',
             'fg'        : 'away_fg',
             'fga'       : 'away_fga',
             '3p'        : 'away_3p',
             '3pa'       : 'away_3pa',
             'ft'        : 'away_ft',
             'fta'       : 'away_fta',
             'or'        : 'away_or',
             'dr'        : 'away_dr',
             'tot'       : 'away_total_reb',
             'a'         : 'away_assists',
             'pf'        : 'away_fouls',
             'st'        : 'away_steals',
             'or'        : 'away_or',
             'to'        : 'away_turnovers',
             'bl'        : 'away_blocks',
             'poss'      : 'away_poss',
             'pace'      : 'away_pace',
             'oeff'      : 'away_off_eff',
             'deff'      : 'away_def_eff',
             'rest_days' : 'away_rest',
             'starting_lineup' : 'away_starter1',
             'unnamed:_35'     : 'away_starter2',
             'unnamed:_36'     : 'away_starter3',
             'unnamed:_37'     : 'away_starter4',
             'unnamed:_38'     : 'away_starter5',
             'crew_referees'   : 'ref_3',
             'main_referee'    : 'ref_1'

             }
    df_road.rename(columns = away, inplace = True)
    

    
    #reset the indexs to merge the files
    df_home.reset_index(inplace = True)
    df_road.reset_index(inplace = True)
                         
    #merge data so game becomes rather than team unit of analysis                      
    new =  pd.concat([df_home,df_road], axis = 1)
  
    #establish data as a date/time variable
    new['date'] = pd.to_datetime(new['date'])
    
    #add cover information
    new['line_cv'] = new.home_score - new.away_score + new.spread
    
    new['cover'] = new['line_cv'].map(lambda x: 1 if x >0 else 0)
    
    #add win and score difference info
    
    
    new['home_win_margin'] = (new['home_score'] - new['away_score'])
    new['win'] = new['home_win_margin'].map(lambda x: 1 if x > 0 else 0)
    new['away_win_margin'] = -new['home_win_margin']
    new['away_win'] = new['win'].map(lambda x: 0 if x == 1 else 1)
    
    
    
    #call second function
    new2 = create_home_mov_ave(new)
    new3 = create_away_mov_ave(new2)
    return new3

### Calulating my dependent variable.  

Line_cv is the amount the game went over or under the spread, and then cover is 1 for a home team spread victory and 0 for an away team spread victory.  This leave us with to possible methods of prediction.  We can use regression to predict the cover amount or we can use categorization to predict which category it fall into.  We also could try and predict individual team scores using regression and use those as a prediction

In [9]:
def create_home_mov_ave(df):
    

    
    #add underscores to teams
    df['teams'] = [str(team).replace(' ', '_') for team in df['teams']]
    
    #loop through teams
    teams = ['Atlanta','Boston','Charlotte','Chicago',
             'Cleveland','Dallas','Denver','Detroit',
             'Golden_State','Houston','Indiana','LA_Clippers',
             'LA_Lakers','Memphis','Miami','Milwaukee',
             'Minnesota','New_Jersey','New_Orleans','New_York',
             'Orlando','Philadelphia','Phoenix','Portland',
             'Sacramento','San_Antonio','Seattle','Toronto',
             'Utah','Washington']
    

    columns = ['1q', '2q', '3q', '4q',
               'ot1', 'ot2', 'ot3', 'ot4', 'home_score', 'min', 'fg', 
               'fga', '3p', '3pa', 'ft','fta', 'or', 'dr', 'tot', 
               'a', 'pf', 'st', 'to', 'pts','home_score',
               'pace', 'oeff', 'deff', 'bl', 'win', 'home_win_margin',
               'away_score']
    #create empty data frame to put in results
    home_vars = pd.DataFrame()
    
    for team in teams:
        #subset by team and sort by date
        df_team = df[df['teams'] == team]
        df.sort_values('date', inplace = True)
      
        #create 5 game moving averages
        for column in columns:
            column_new = 'mov_5_' + column
            df_team[column_new] = df_team[column].rolling(7).mean().shift(1)
        
            #three game moving average
            column_3 = 'mov_3_' + column
            df_team[column_3] = df_team[column].rolling(2).mean().shift(1)    
        
        
        #create average home scores and win totals
        home_win_pcts = []
        home_win_pct = 0
        
        #create average home win margin scores
        home_ave_win_margins = []
        home_ave_win_margin = 0
        
        for i in range(len(df_team['win'])):
            
            #cumulative win percentages
            home_win_pct += df_team.iloc[i, 92]
            home_win_pcts.append(home_win_pct/(i+1))
            
            #average home win margin
            home_ave_win_margin += df_team.iloc[i,91 ]
            home_ave_win_margins.append(home_ave_win_margin/(i+1))
       
        #assign the home win percents
        df_team['home_win_pct'] = home_win_pcts    
        df_team['home_win_pct'] = df_team.home_win_pct.shift(1)
        
        #Assign average winning margin
        df_team['home_ave_win_margin'] = home_ave_win_margins
        df_team['home_ave_win_margin'] = df_team.home_ave_win_margin.shift(1)
        
        home_vars = home_vars.append(df_team)
    return home_vars 



In [10]:
def create_away_mov_ave(df):
    

    
    #add underscores to teams
    df['away_team'] = [str(team).replace(' ', '_') for team in df['away_team']]
    
    #loop through teams
    teams = ['Atlanta','Boston','Charlotte','Chicago',
             'Cleveland','Dallas','Denver','Detroit',
             'Golden_State','Houston','Indiana','LA_Clippers',
             'LA_Lakers','Memphis','Miami','Milwaukee',
             'Minnesota','New_Jersey','New_Orleans','New_York',
             'Orlando','Philadelphia','Phoenix','Portland',
             'Sacramento','San_Antonio','Seattle','Toronto',
             'Utah','Washington']
    

    columns = ['away_1q', 'away_2q', 'away_3q', 'away_4q',
               'away_ot1', 'away_ot2', 'away_ot3', 'away_ot4', 
               'away_score', 'away_min', 'away_fg', 'away_fga', 
               'away_3p', 'away_3pa', 'away_ft','away_fta',
               'away_or', 'away_dr', 'away_total_reb', 
               'away_assists', 'away_fouls', 'away_steals', 
               'away_turnovers','away_blocks', 
               'away_off_eff',
               'away_def_eff', 'away_win', 'away_win_margin']
    #create empty data frame to put in results
    away_vars = pd.DataFrame()
    
    for team in teams:
        
        df_team = df[df['away_team'] == team]
        df.sort_values('date', inplace = True)
        for column in columns:
            
            #five game moving average
            column_new = 'mov_5_' + column
            df_team[column_new] = df_team[column].rolling(7).mean().shift(1)
            
            #three game moving average
            column_3 = 'mov_3_' + column
            df_team[column_3] = df_team[column].rolling(2).mean().shift(1)
        
        #create average away win totals
        away_win_pcts = []
        away_win_pct = 0
        
        #create average home win margin scores
        away_ave_win_margins = []
        away_ave_win_margin = 0
        
        for i in range(len(df_team['win'])):
            
            #cumulative win percentages
            away_win_pct += df_team.iloc[i, 94]
            away_win_pcts.append(away_win_pct/(i+1))
            
            #average home win margin
            away_ave_win_margin += df_team.iloc[i,93 ]
            away_ave_win_margins.append(away_ave_win_margin/(i+1))
       
        #assign the home win percents
        df_team['away_win_pct'] = away_win_pcts    
        df_team['away_win_pct'] = df_team.away_win_pct.shift(1)
        
        #Assign average winning margin
        df_team['away_ave_win_margin'] = away_ave_win_margins
        df_team['away_ave_win_margin'] = df_team.away_ave_win_margin.shift(1)
        
        
        away_vars = away_vars.append(df_team)
    
        
    return away_vars



In [11]:
nba2006_trans = merge_home_away(nba2006)

/Users/bryancombs/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:42: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/Users/bryancombs/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:51: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/Users/bryancombs/anaconda3/lib/python3.6/site-packages/pandas/core/frame.py:3027: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  return super(DataFrame, self).rename(**kwargs)
/Users/bryancombs/anaconda3/lib/python3.6/site-packages/ipyker

TypeError: unsupported operand type(s) for +=: 'int' and 'str'

In [ ]:
nba2006_trans.T

In [ ]:
def merge_years(df1,df2):
    return pd.concat([df1,df2], axis = 0)
        

In [ ]:
nba2006_trans.home_win_margin.value_counts()

In [ ]:

nba2007_trans = merge_home_away(nba2007)
nba2008_trans = merge_home_away(nba2008)
nba2009_trans = merge_home_away(nba2009)
nba2010_trans = merge_home_away(nba2010)
nba2011_trans = merge_home_away(nba2011)
nba2012_trans = merge_home_away(nba2012)
nba2013_trans = merge_home_away(nba2013)
nba2014_trans = merge_home_away(nba2014)

In [ ]:
nba2013_trans.drop(columns = ['to_to'])
nba2014_trans.drop(columns = ['to_to']) 


In [ ]:
nba2013_trans.head().T


In [ ]:
nba_combined = merge_years(nba2006_trans,nba2007_trans)


In [ ]:
nba_combined = merge_years(nba_combined,nba2008_trans)
nba_combined = merge_years(nba_combined,nba2009_trans)
nba_combined = merge_years(nba_combined,nba2010_trans)
nba_combined = merge_years(nba_combined,nba2011_trans)
nba_combined = merge_years(nba_combined,nba2012_trans)


In [ ]:
nba_combined.columns[0:50]

In [ ]:
nba2013_trans.columns[0:50]

In [ ]:
nba_combined = merge_years(nba_combined,nba2013_trans)
nba_combined = merge_years(nba_combined,nba2014_trans)

### Dropping columns that are no longer necessary.  

The NBA data that relates to the particular game that has just been played is not longer necessary so we can drop those columns to make sure that they are not used in the analysis as we do not have access to that data.   

In [ ]:
nba_combined.drop(columns = ['away_1q', 'away_2q', 'away_3q', 'away_4q',
               'away_ot1', 'away_ot2', 'away_ot3', 'away_ot4', 
               'away_min', 'away_fg', 'away_fga', 
               'away_3p', 'away_3pa', 'away_ft','away_fta',
               'away_or', 'away_dr', 'away_total_reb', 
               'away_assists', 'away_fouls', 'away_steals', 
               'away_turnovers','away_blocks', 
               'away_off_eff','away_def_eff', '1q', '2q', '3q', '4q',
               'ot1', 'ot2', 'ot3', 'ot4', 'min', 'fg', 
               'fga', '3p', '3pa', 'ft','fta', 'or', 'dr', 'tot', 
               'a', 'pf', 'st', 'to', 'pts', 'poss',
               'pace', 'oeff', 'deff', 'bl',
               'crew_referee', 'index', 'home_score',
               'starting_lineups', 'home_starter2', 'home_starter3',
               'home_starter4', 'away_score', 'win', 'line_cv'             
                            ], inplace = True)

In [ ]:
nba_combined.to_csv('./data/nba_combined.csv', index = False)

In [ ]:
nba_combined.columns[0:100]